<style>
    h1{
        text-align:center;
    }
    h2{
        margin-left:5rem;
        text-decoration:underline;
    }
</style>

<h1>Artificial Intelligence Algorithms - MESIIN476023</h1>

<h2>Problem Description</h2>

<p>The problem at hand is to optimize the production of biscuits from a single roll of dough in a biscuit manufacturing factory for the Christmas season. The factory aims to produce four types of biscuits with different sizes and prices all while using the available roll of dough. The goal is to maximize biscuit production while ensuring the highest possible profit.</p>

<h2>Algorithms</h2>

<p>For solving this problem we chose to use 2 different algorithms: <b>Genetic Algorithm (GA)</b> and <b>Greedy Search Algorithm</b>. The former is from the local search algorithms and the latter a heuristic algorithm. Let's give a small explanation of each algorithm before proceeding by using the course slides.</p>

<h3>Genetic Algorithm (GA)</h3>
<p>Genetic algorithms (or GA) are inspired by natural evolution and are particularly useful in optimization and search problems with large state spaces.
Given a problem, algorithms in the domain make use of a population of solutions (also called states), where each solution/state represents a feasible solution. At each iteration (often called generation), the population gets updated using methods inspired by biology and evolution, like crossover, mutation and natural selection.</p>

<h3>Greedy Search Algorithm</h3>
<p>Greedy makes locally optimal choices at each step with the aim of finding a solution. It starts with an initial solution and iteratively expands it by selecting the best available option at each decision point.</p>

<style>
    h1{
        text-align:center;
    }
    h2{
        margin-left:5rem;
        text-decoration:underline;
    }
</style>

<h2>Genetic Algorithm (GA)</h2>

Let's start with the implementation of the Genetic Algorithm (GA). For this we implemented two classes <b>Dough</b> and <b>Biscuits</b> that we saved in other <i>.py</i> files for a more object-oriented and organized code. We know that the implementation of the GA works in the following way:

1. Initialization: Start with a population of randomly generated routes.
2. Define the fitness function
3. Selection: Select the best routes based on their fitness (shortest distances).
4. Crossover (Recombination): Create new routes by combining pairs of the best routes.
5. Mutation: Introduce random changes into the new routes.
6. Replacement: Replace the old population with the new population of routes.
7. Termination: Stop if a maximum number of iterations is reached or if the best route hasn't improved for a certain number of iterations.

But first we will start by importing the two classes and libraries that will be needed for solving this problem.

In [335]:
import numpy as np
import dough
import biscuits
import random

Now that we have the corresponding classes and librairies that will be useful to solve the problem, we can create instances of the <b>Dough</b> and <b>Biscuits</b> classes based on the benchmark that was given by the company.

In [336]:
# The length of the roll of dough is set to 500 units.
LENGTH = 500

dough = Dough(LENGTH)
dough.load_defects("defects.csv")

# The biscuit manufacturing factory aims to produce 4 types of biscuits:

biscuit0 = Biscuit(0, 4, 6, {'a' : 4, 'b' : 2, 'c' : 3})
biscuit1 = Biscuit(1, 8, 12, {'a' : 5, 'b' : 4, 'c' : 4})
biscuit2 = Biscuit(2, 2, 1, {'a' : 1, 'b' : 2, 'c' : 1})
biscuit3 = Biscuit(3, 5, 8, {'a' : 2, 'b' : 3, 'c' : 2})

biscuits = [biscuit0, biscuit1, biscuit2, biscuit3]

Let's move on to the implementation of the genetic algorithm:

<h3>1. Initialization</h3>

In [337]:
"""
    In the initialization process, we will assign a random number of each type of biscuit to the roll of dough. 
    Biscuit0, Biscuit1, Biscuit2, and Biscuit3 are assigned to the roll of dough with the numbers of 0, 1, 2, and 3, respectively.
    The types are put in the list with the corresponding sizes. The number -1 is assigned to the empty space on the roll of dough.
    --------------------------------------------------
    Parameters:
        population_size: The number of individuals in the population.
        biscuits: The list of biscuit types.
        dough: The roll of dough.
    --------------------------------------------------
    Returns:
        population: A list of individuals in the population.
"""
def initial_population(population_size, biscuits, dough):
    population = []

    for _ in range(population_size):
        individual = []
        position = 0

        while position < dough.length:
            selected_biscuit = random.choice(biscuits)
            next_position = position + selected_biscuit.size

            # Check if the biscuit fits in the remaining dough length, if not we fill the remaining dough with -1
            if next_position > dough.length:
                individual.extend([-1] * (dough.length - position))
                break

            # We count the number of defects for each class in the position that the biscuit will be placed
            defect_count = {defect_class: 0 for defect_class in ['a', 'b', 'c']}
            for p in range(position, next_position):
                if p in dough.defects:
                    defect_count[dough.defects[p]] += 1

            # We check to see if the number of defects in the position that the biscuit will be placed is less than the maximum allowed number of defects for the biscuit
            if not selected_biscuit.check_defect_threshold(defect_count):
                individual.extend([selected_biscuit.id] * selected_biscuit.size)
                position = next_position
            else:
                individual.append(-1)
                position += 1

        # Since there was a problem before where the individuals had different lengths not corresponding to the benchmark in this part we ensure that individual length matches dough length
        individual = individual[:dough.length]
        population.append(individual)

    return population

<h3>2. Fitness function</h3>

In our fitness function we will try to maximize the value of the solution which will be the sum of the values of the individual biscuits placed on it. 

$$
max \quad f(x) \quad with \quad f(x) = \sum_{i=1}^{n} x_i \quad and \quad x_i \quad the \quad value/price \quad of \quad each \quad biscuit \quad type.
$$

In [338]:
"""
    Calculate the fitness of an individual in the population. To do so, we will skip if the current position is empty (-1). If the position is not empty, we check the biscuit
    type and calculate the value of that biscuit before moving on to the next. We will add the value of the biscuit to the total value and move to the next position.
    ----------------
    Parameters:
        individual: The individual from the population, a list of biscuit types placed on the dough that represents a possible solution.
        biscuits: List of biscuit objects, each with a specific value.
    ----------------
    Returns:
        int: The total value of all biscuits in the individual solution.
"""

def fitness(individual, biscuits):
    total_value = 0
    i = 0
    while i < len(individual):
        biscuit_type = individual[i]
        
        if biscuit_type == -1:
            i += 1
            continue
        biscuit = biscuits[biscuit_type]
        if all(biscuit_type == individual[j] for j in range(i, min(i + biscuit.size, len(individual)))):
            total_value += biscuit.value
            i += biscuit.size
        else:
            i += 1

    return total_value

<h3>3. Selection phase</h3>

For the selection phase, individuals are selected from the population so that they can be the parents for the next generation based on their fitness. Individuals with higher fitness have a higher probability of being selected, mimicking the natural selection process. We will use elitism and the roulette wheel selection for this part.

In [339]:
"""
    Select individuals for the next generation using elitism and fitness-proportional selection. We first sort the population by fitness in descending order. Then, we select the 
    elite individuals with the help of elite_size. The total fitness is calculated by summing the fitness of all individuals in the population.
    ----------------
    Parameters:
        fitness: function to calculate the fitness of an individual.
        population: The current population of individuals.
        n_population: The total number of individuals in the population.
        elite_size: The number of elite individuals to carry over to the next generation.
    ----------------
    Returns:
        list: The new generation of individuals.
"""

def select_elitismRoulette(fitness, population, n_population, elite_size, biscuits):
    sorted_population = sorted(population, key=lambda individual: fitness(individual, biscuits), reverse=True)
    elites = sorted_population[:elite_size]

    # Roulette wheel selection
    total_fitness = sum(fitness(individual, biscuits) for individual in sorted_population)
    selection_probabilities = [fitness(individual, biscuits) / total_fitness for individual in sorted_population]
    selected_via_roulette = []
    for _ in range(n_population - elite_size):
        selected_via_roulette.append(sorted_population[roulette_wheel_choice(selection_probabilities)])

    # Combine elites and individuals selected via roulette wheel
    new_generation = elites + selected_via_roulette

    return new_generation

def roulette_wheel_choice(selection_probabilities):
    cumulative_probabilities = [sum(selection_probabilities[:i+1]) for i in range(len(selection_probabilities))]
    random_number = random.random()

    for index, cumulative_probability in enumerate(cumulative_probabilities):
        if random_number < cumulative_probability:
            return index

<h3>4. Crossover</h3>

Now that the selection has been made, individuals can produce and make a child. This offspring bears characteristics from both of its parents. There are many ways we can implement this crossover. Here we will try the <b>uniform crossover</b>. 

In [340]:
"""
In the uniform crossover, we will iterate over the parents and swap the elements with a probability of 0.5.
"""
def uniform_crossover(parent1, parent2):
    child1, child2 = [], []
    for i in range(len(parent1)):
        if random.random() < 0.5:
            child1.append(parent1[i])
            child2.append(parent2[i])
        else:
            child1.append(parent2[i])
            child2.append(parent1[i])
    return child1, child2

<h3>5. Mutation</h3>

The mutation function will randomly alter the type of biscuit at one or more positions in an individual's arrangement. 

In [341]:
"""
    Mutate an individual by randomly changing the biscuit type at some positions.
    ----------------
    Parameters:
        individual: The individual to be mutated.
        mutation_rate: The probability of each gene (biscuit position) being mutated.
        biscuits: List of possible biscuit types.
    ----------------
    Returns:
        list: The mutated individual.
    """

def mutate(individual, mutation_rate, biscuits):
    mutated_individual = individual.copy()
    for i in range(len(mutated_individual)):
        if random.random() < mutation_rate:
            new_biscuit_type = random.choice(range(len(biscuits)))
            mutated_individual[i] = new_biscuit_type
    return mutated_individual

<h3>Implementation of the GA</h3>

To create a genetic algorithm (GA) for the biscuit manufacturing problem, we need to integrate all the components mentioned previouslt: initialization, fitness evaluation, selection, crossover, and mutation. The parameters of the method are the following: biscuits, population_size, generations, mutation_rate, and elite_size. 

In [342]:
def genetic_algorithm(biscuits, population_size, generations, mutation_rate, elite_size):
    # Initialize the population
    population = initial_population(population_size, biscuits, dough)
    best_individual = max(population, key=lambda ind: fitness(ind, biscuits))

    best_fitness = fitness(best_individual, biscuits)

    # Evolution loop
    for _ in range(generations):
        new_population = []

        # Apply crossover and mutation
        for _ in range(population_size // 2): 
            parent1, parent2 = select_elitismRoulette(fitness, population, population_size, elite_size, biscuits)[0], \
                               select_elitismRoulette(fitness, population, population_size, elite_size, biscuits)[0]

            # Perform crossover
            child1, child2 = uniform_crossover(parent1, parent2)  # or two_point_crossover or uniform_crossover
            
            # Perform mutation
            child1 = mutate(child1, mutation_rate, biscuits)
            child2 = mutate(child2, mutation_rate, biscuits)

            new_population.extend([child1, child2])

        # Replace old population with new population
        population = new_population
        current_best_individual = max(population, key=lambda ind: fitness(ind, biscuits))
        current_best_fitness = fitness(current_best_individual, biscuits)

        # Update best solution found so far
        if current_best_fitness > best_fitness:
            best_fitness = current_best_fitness
            best_individual = current_best_individual

    print(f"Final Best Fitness: {best_fitness}")
    return best_individual

best_solution = genetic_algorithm(biscuits, population_size=100, generations=1000, mutation_rate=0.05, elite_size=10)
print("Best Solution:", best_solution)

Final Best Fitness: 724
Best Solution: [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 3, 0, 0, 0, 0, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, -1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, -1, -1, 0, 0, 0, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 0, 0, 0, 0, 3, 3, 3, 3, 3, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3,

At the end we found a best fitness score of 724 but the algorithm took 42 minutes to run for a population of 100 which is a very long time. Let us see what we can find with the next algorithm: <b>Greedy Search Algorithm</b>.

<style>
    h1{
        text-align:center;
    }
    h2{
        margin-left:5rem;
        text-decoration:underline;
    }
</style>

<h2>Greedy Search</h2>



In [346]:
import pandas as pd

# Load defects data
defects_df = pd.read_csv('defects.csv')
defects = defects_df.groupby('class')['x'].apply(list).to_dict()

# Biscuits definition
biscuits = [
    {"id": 0, "length": 4, "value": 6, "defects": {'a': 4, 'b': 2, 'c': 3}},
    {"id": 1, "length": 8, "value": 12, "defects": {'a': 5, 'b': 4, 'c': 4}},
    {"id": 2, "length": 2, "value": 1, "defects": {'a': 1, 'b': 2, 'c': 1}},
    {"id": 3, "length": 5, "value": 8, "defects": {'a': 2, 'b': 3, 'c': 2}},
]

DOUGH_LENGTH = 500

*   Biscuit Sorting: The biscuits are sorted in descending order of their value-to-size ratio.

*   Placement Logic: The place_biscuits function tries to place each biscuit on the dough roll starting from the highest value-to-size ratio, checking for defect constraints.

*   Position Iteration: The algorithm iterates over the dough roll positions, trying to fit biscuits and skipping positions where placement is not possible due to defects or size constraints.

In [347]:
# Sort biscuits based on value-to-size ratio
biscuits.sort(key=lambda x: x['value'] / x['length'], reverse=True)

# Function to check if a biscuit can be placed at a position
def can_place_biscuit(biscuit, position):
    if position + biscuit['length'] > DOUGH_LENGTH:
        return False
    for defect_class, max_allowed in biscuit['defects'].items():
        defects_in_range = [d for d in defects.get(defect_class, []) if position <= d < position + biscuit['length']]
        if len(defects_in_range) > max_allowed:
            return False
    return True

# Function to place biscuits on the dough
def place_biscuits():
    placement = []
    position = 0
    while position < DOUGH_LENGTH:
        placed = False
        for biscuit in biscuits:
            if can_place_biscuit(biscuit, position):
                placement.append((biscuit['id'], position))
                position += biscuit['length']
                placed = True
                break
        if not placed:
            position += 1  # Move to the next position if no biscuit can be placed
    return placement

# Get the placement of biscuits
biscuit_placement = place_biscuits()

In [348]:
total_value = sum(biscuits[biscuit_id]['value'] for biscuit_id, _ in biscuit_placement)
print(f"Total value of placed biscuits: {total_value}")
print("Biscuit placements (id, start position):", biscuit_placement)

Total value of placed biscuits: 484
Biscuit placements (id, start position): [(1, 0), (1, 8), (3, 16), (0, 21), (0, 25), (3, 29), (2, 34), (0, 36), (0, 40), (0, 44), (2, 48), (0, 50), (3, 54), (3, 59), (0, 64), (3, 68), (0, 73), (3, 77), (1, 82), (0, 90), (2, 95), (0, 98), (3, 102), (0, 107), (0, 111), (0, 118), (0, 122), (3, 126), (3, 131), (0, 136), (0, 140), (3, 144), (3, 149), (2, 154), (3, 158), (2, 164), (3, 167), (3, 172), (0, 177), (2, 181), (2, 185), (1, 187), (3, 195), (3, 200), (3, 205), (3, 210), (3, 215), (3, 220), (3, 225), (3, 230), (3, 235), (0, 242), (0, 246), (3, 250), (3, 255), (3, 260), (3, 265), (3, 270), (0, 275), (0, 279), (3, 283), (3, 288), (1, 293), (1, 301), (3, 309), (0, 314), (3, 318), (3, 323), (0, 328), (2, 332), (3, 334), (3, 339), (3, 344), (3, 349), (1, 354), (0, 362), (0, 366), (0, 370), (0, 374), (3, 378), (3, 383), (3, 388), (3, 393), (0, 398), (3, 402), (3, 407), (3, 412), (3, 417), (3, 422), (0, 427), (0, 431), (0, 435), (3, 439), (0, 444), (3, 44

The heuristic used here is the value-to-size ratio, but other heuristics can also be experimented with. In the greedy algorithm, we have a much lower fitness score compared to the Genetic Algorithm (GA) but the execution speed for the Greedy Algorithm is much faster. 

<style>
    h1{
        text-align:center;
    }
    h2{
        margin-left:5rem;
        text-decoration:underline;
    }
</style>

<h2>Constraint Satisfaction Problems (CSPs)</h2>

To add a little bit more to the project and since the greedy never performed that well, we also decided to add a solution to the problem using the Constraint Satisfaction Problems (CSPs). We though that it would be interesting to add this algorithm as there were complex constraints that needed to be handled such as not having any overlaping biscuits, that the sum of the size of each biscuits should not exceed the length of the roll of dough, and the threshold constraint. 

We first start by importing the OR-Tools and other required libraries such as pandas to be able to read the <i>defects.csv</i> file.

In [343]:
import pandas as pd
from ortools.sat.python import cp_model

Then for the implementation of the problem using CSP we do the following:
1. Define the model using the CpModel
2. Load the defects using pandas and define the 4 types of biscuits stated by the manufacturers
3. We define a variable for each biscuit type at each possible position
4. We then add the necessary constraints. In the benchmark written by the company it was said to have no overlapping biscuits and the threshold for the maximum number of defects per class type should not be passed.
5. Since we want to maximize the total value of the biscuits placed on the roll we define the objective function
6. Then we solve the model using the OR-Tools solver method
7. Finally we output the results which is the fitness score and the position of each biscuit type on the roll of dough. 

In [344]:
model = cp_model.CpModel()

defects_df = pd.read_csv('defects.csv')
biscuits = [
    {'size': 4, 'value': 6, 'defects': {'a': 4, 'b': 2, 'c': 3}},
    {'size': 8, 'value': 12, 'defects': {'a': 5, 'b': 4, 'c': 4}},
    {'size': 2, 'value': 1, 'defects': {'a': 1, 'b': 2, 'c': 1}},
    {'size': 5, 'value': 8, 'defects': {'a': 2, 'b': 3, 'c': 2}}
]


max_biscuits = LENGTH // min(b['size'] for b in biscuits)
biscuit_vars = {}
for i, biscuit in enumerate(biscuits):
    for pos in range(LENGTH - biscuit['size'] + 1):
        biscuit_vars[(i, pos)] = model.NewBoolVar(f'biscuit_{i}_pos_{pos}')


for pos in range(LENGTH):
    model.Add(sum(biscuit_vars[(i, p)] for i, biscuit in enumerate(biscuits) for p in range(max(0, pos - biscuit['size'] + 1), min(LENGTH - biscuit['size'], pos) + 1)) <= 1)


for biscuit_index, biscuit in enumerate(biscuits):
    for defect_class in ['a', 'b', 'c']:
        for pos in range(LENGTH - biscuit['size'] + 1):
            defect_count = defects_df[(defects_df['x'] >= pos) & (defects_df['x'] < pos + biscuit['size']) & (defects_df['class'] == defect_class)].shape[0]
            model.Add(biscuit_vars[(biscuit_index, pos)] * defect_count <= biscuit['defects'][defect_class])


model.Maximize(
    sum(biscuit_vars[(i, pos)] * biscuit['value'] for i, biscuit in enumerate(biscuits) for pos in range(LENGTH - biscuit['size'] + 1))
)


solver = cp_model.CpSolver()
status = solver.Solve(model)


if status == cp_model.OPTIMAL:
    print("Maximum value:", solver.ObjectiveValue())
    for i, biscuit in enumerate(biscuits):
        for pos in range(LENGTH - biscuit['size'] + 1):
            if solver.BooleanValue(biscuit_vars[(i, pos)]):
                print(f"Biscuit {i} placed at position {pos}")
else:
    print("No solution found.")

Maximum value: 760.0
Biscuit 0 placed at position 9
Biscuit 0 placed at position 36
Biscuit 0 placed at position 46
Biscuit 0 placed at position 50
Biscuit 0 placed at position 54
Biscuit 0 placed at position 58
Biscuit 0 placed at position 75
Biscuit 0 placed at position 84
Biscuit 0 placed at position 93
Biscuit 0 placed at position 104
Biscuit 0 placed at position 114
Biscuit 0 placed at position 118
Biscuit 0 placed at position 122
Biscuit 0 placed at position 136
Biscuit 0 placed at position 140
Biscuit 0 placed at position 144
Biscuit 0 placed at position 148
Biscuit 0 placed at position 158
Biscuit 0 placed at position 162
Biscuit 0 placed at position 172
Biscuit 0 placed at position 186
Biscuit 0 placed at position 200
Biscuit 0 placed at position 214
Biscuit 0 placed at position 218
Biscuit 0 placed at position 242
Biscuit 0 placed at position 246
Biscuit 0 placed at position 275
Biscuit 0 placed at position 279
Biscuit 0 placed at position 314
Biscuit 0 placed at position 336

By using the CSP we notice that the run time was much faster than the genetic algorithm. In addition to that, the fitness score is higher, here we have 760 and in the GA we had 724. Finally it seems that in this solution there is no placement for the biscuit of type 2. 